In [1]:
# if you want to have a pop up sign in method instead please uncomment  on ee.Authenticate and ee.Initialize, but comment on below

import ee

# Trigger the authentication flow. if you want to user json, please comment this
ee.Authenticate()
# Initialize the library.
ee.Initialize()

# here we try to use json api information instead (service account) - comment below if you use above methode
#import ee
import math
from datetime import datetime

# uncomment this if you want to use json instead
#service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, './bukit30project-4d92e5b46ea7.json')
#ee.Initialize(credentials)

Enter verification code: 4/1AfJohXnhq2ahjRJ0nxOM2VjRtwXAtpGRXOo-m6a3xiDlIGIqYxSu4bquvSQ

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


In [2]:
import geemap

# AOI File (SHP) - added shp_local: please make a folder in the main directory: shp_local and put here as an input (example) - shp local not publish to github
AOIt_shp = './shp_local/AOI_B30.shp'
AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

AOI = AOIt_shp_plot

In [3]:
from osi.image_collection.main import ImageCollectionComposite

# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollectionComposite(AOI=AOI, date_start_end=['2022-1-1',"2022-12-31"], cloud_cover_threshold = 40)


In [4]:
image_landsat_collection_mask = classInputCollection.merging_collection_landsat()

selecting LANDSAT images


In [5]:
image_landsat_cloudless_composite = image_landsat_collection_mask.median().clip(AOI)

In [6]:
Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)

In [7]:
# Map.addLayer(image_landsat_cloudless_composite,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, 'landsat_for_FCD_2023_uncorrected')

In [8]:
image_planet_clip = classInputCollection.merging_collection_planet('asia').median().clip(AOI)

In [9]:
# ADDING VISUAL FOR CHECKING AND VERIFY THE RESULT
visPlanet_veg = {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5}
truePlanet_veg = {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}

# Map.addLayer(image_planet_clip, visPlanet_veg , 'NICFI Cloudless Image - RNB', False)
# Map.addLayer(image_planet_clip, truePlanet_veg , 'NICFI Cloudless Image - RGB')

In [10]:
from osi.image_collection.topographic_correction import doPhysicalCorrection

image_corrected_composite_landsat = image_landsat_collection_mask.map(lambda image: doPhysicalCorrection(image, 
                                                                                                         type_DEM='srtm', 
                                                                                                         I_satellite='LANDSAT')) \
                                                                                                         .median().clip(AOI)

In [11]:
Map.addLayer(image_corrected_composite_landsat,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, f'landsat_for_FCD_2023_corrected')
    

In [12]:
from osi.spectral_indices.dn_spectral import SpectralAnalysis

classImageSpectral = SpectralAnalysis(image_corrected_composite_landsat, AOI = AOI, I_satellite= 'LANDSAT', pca_scaling = 1, tileScale = 1)


In [13]:
avi_image = classImageSpectral.AVI_func()

In [14]:
Map.addLayer(avi_image,{},'avi')

In [15]:
Map.addLayerControl()

In [16]:
Map

Map(center=[-1.0232057340644143, 102.432211047781], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
from lib_module.spectral_indices import main

In [ ]:
adding = main.adding_all(5)

In [ ]:
adding